# `importnb` test specification

this notebook is written to test many of the features of `importnb`.

these features in this notebook test:
* basic finding and loading
* filtering magics, classes and functions
* lazy loading
* the command line interface

In [ ]:
from __future__ import annotations

import sys

In [ ]:
def report_env(when: str, scope: dict[str, Any]) -> None:
    lines = [f"## {when}", ""]
    mods = sorted(
        m for m in sorted(sys.modules, key=str.lower) if "." not in m and not m.startswith("_")
    )
    lines += [
        f"  - [{'x' if v else ' '}] {k}" for k, v in sorted(scope.items()) if k.startswith("IS_")
    ]
    lines += ["", f"> {len(mods)} modules: `{'` & `'.join(mods)}`", ""]
    print("\n".join(lines), file=sys.stderr)


report_env("before sniffing", locals())

In [ ]:
from pathlib import Path
from typing import TYPE_CHECKING, Any

In [ ]:
if TYPE_CHECKING:
    from argparse import ArgumentParser

In [ ]:
IS_MAIN = __name__ == "__main__"
IS_FILE = "__file__" in locals()
IS_PYTEST = "pytest" in sys.modules
IS_DOIT = "doit" in sys.modules
IS_WIN = sys.platform == "win32"
IS_WHERE = Path(__file__).as_posix() if IS_FILE else ""
IS_IPYTHON = "IPython" in sys.modules

report what we can tell about the environment

In [ ]:
report_env("before importnb", locals())

detect additional values which might change based on importing `importnb` 

In [ ]:
from importnb import get_ipython

IS_IPYTHON = get_ipython() is not None
report_env("after importnb", locals())

## a sentinel for execution

the `slug` below is used to measure that a module has been executed,
we specifically use this expression to measure the lazy importing system.

In [ ]:
SLUG = "i was printed from {IS_WHERE} \
and my name is {__name__}"
print(SLUG.format(**locals()))

## implicit markdown docstrings

there is a strict separation of code and non-code in notebooks.
to encourage more/better documentation `importnb` will use a markdown
cell preceeding a function as a docstring. 
as a result, the `function_with_a_markdown` docstring will have this markdown cell for a value.

In [ ]:
def function_with_a_markdown_docstring() -> None:
    return  # function_with_a_markdown has a docstring defined by the preceeding markdown cell

the same convention holds for classes and async functions.

In [ ]:
class class_with_a_markdown_docstring: ...  # my docstring is the cell above.

this is not a docstring for `class_with_a_string` because it defines its own.

In [ ]:
class class_with_a_python_docstring:
    """when a class defines its own docstring the preceeding cell is ignored."""

## cell magics

In [ ]:
%%python
print("i'm only show when cell magics are active.")

In [ ]:
if IS_IPYTHON:
    SLUG = "i'm only show when cell magics are active."
    if IS_WIN:
        SLUG += "\n"
else:
    SLUG = f"this was printed from the module named {__name__}"
    print(SLUG)

## notebooks as scripts

the main block is a python convention we can apply in notebooks imported by importnb.

In [ ]:
def get_parser() -> ArgumentParser:
    from argparse import REMAINDER, ArgumentParser

    parser = ArgumentParser("test_parser")
    parser.add_argument("--", nargs=REMAINDER, dest="args")
    parser.add_argument("-h", "--help", "i shadow the importnb help argument", action="store_true")
    return parser

In [ ]:
def main(argv: list[str] | None = None) -> int:
    parser = get_parser()
    print("i am parsing", argv)
    ns = parser.parse_args(argv)
    print("the parser namespace is", ns)
    return 0

### notebooks as `doit` tasks

[`doit`](https://pydoit.org/) is powerful alternative to makefiles for running development tasks.
the `importnb` command line provides support for `doit` conventions, but does not provide the dependency;
you the `doit` are responsible for that.

this the docstring for the `echo` task that echos hello.

In [ ]:
def task_echo() -> dict[str, Any]:
    return dict(actions=["echo hello"])

In [ ]:
if IS_MAIN and not (IS_PYTEST or IS_DOIT):
    # run this notebook like it is a cli
    sys.exit(main(sys.argv[1:]))

## data loaders

data loaders can import other file formats. we can hide loading logic underneath `import` statements.

In [ ]:
if IS_IPYTHON and not IS_WHERE:
    from importnb import loaders

    display(loaders.Json.load_file("Untitled42.ipynb"))